<a href="https://colab.research.google.com/github/chen2369/data-course-sample/blob/main/Collaborative%20filtering%20%E6%8E%A8%E8%96%A6%E7%B3%BB%E7%B5%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code

## 基礎建設

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2022-01-07 17:28:06--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv’

All_Beauty.csv      100%[===================>]  14.78M  20.0MB/s    in 0.7s    

2022-01-07 17:28:07 (20.0 MB/s) - ‘All_Beauty.csv’ saved [15499476/15499476]

--2022-01-07 17:28:07--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz’

meta_All_Beauty.jso 100%[===================>]   9.85M  15.1MB/s    in 0.7s    

202

In [ ]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [ ]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [ ]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

metadata -> metadata_clean

In [ ]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

In [ ]:
import numpy as np

def emptylist_na(cell):
    try:
        if len(cell) == 0:
            return np.nan
        else:
            return cell
    except:
        return cell
metadata_na = metadata.applymap(emptylist_na)

In [ ]:
metadata_na.isna().sum().sort_values(ascending=False).to_frame(name='Number of NA').T

,category,tech2,fit,tech1,date,feature,similar_item,also_buy,also_view,price,description,imageURLHighRes,imageURL,brand,rank,details,title,main_cat,asin
Number of NA,32892,32892,32892,32882,32873,32623,31588,26295,24760,21433,17773,16351,16351,15673,377,132,1,0,0


In [ ]:
print('# main_cat:', metadata['main_cat'].unique())
print('# similar_item:', metadata['similar_item'].unique())
print('# details:', metadata['details'].sample(5))

# main_cat: ['All Beauty']
# similar_item: [''
 ' class="a-bordered a-horizontal-stripes  a-spacing-extra-large a-size-base comparison_table">\n\n\n\n            \n            \n            \n            \n            \n            <tr class="comparison_table_image_row">\n                <td class="comparison_table_first_col"></td>\n\n\n                <th class="comparison_image_title_cell" role="columnheader">\n                    <div class="a-row a-spacing-top-micro">\n                        <center>\n                             <img alt="No7 Lift &amp; Luminate Triple Action Serum 50ml by Boots" src="https://images-na.ssl-images-amazon.com/images/I/31gXVsxM7FL._SL500_AC_SS350_.jpg" id="comparison_image">\n                        </center>\n                    </div>\n                    <div class="a-row a-spacing-top-small">\n                        <div id="comparison_title" class="a-section a-spacing-none">\n                            <span aria-hidden="true" class="a-size-b

In [ ]:
metadata.head(1)

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]


In [ ]:
(metadata.asin.value_counts()!=1).sum()

404

In [ ]:
metadata_clean = metadata[['asin', 'rank', 'brand', 'price', 'also_view', 'also_buy']]
metadata_clean = metadata_clean.drop_duplicates('asin')
metadata_clean.head(1)

,asin,rank,brand,price,also_view,also_buy
0,6546546450,"2,938,573 in Beauty & Personal Care (",idea village,,[],[]


In [ ]:
metadata_clean['price'] = metadata_clean['price'].str.replace('$', '')
metadata_clean['price'] = pd.to_numeric(metadata_clean['price'], errors = 'coerce')

In [ ]:
metadata_clean['cate'] = metadata_clean['rank'].str.split(' in ').str[1]
metadata_clean['cate'] = metadata_clean['cate'].str.replace('amp;', '')
metadata_clean['cate'] = metadata_clean['cate'].str.replace(' \(', '')
metadata_clean['cate'].unique()

array(['Beauty & Personal Care', nan, 'Grocery & Gourmet Food',
       'Health & Household', 'Sports & Outdoors', 'Toys & Games', 'Baby',
       'Clothing, Shoes & Jewelry', 'Home & Kitchen', 'Automotive',
       'Tools & Home Improvement'], dtype=object)

In [ ]:
metadata_clean = metadata_clean[metadata_clean['cate'] == 'Beauty & Personal Care']

In [ ]:
metadata_clean['rank'] = metadata_clean['rank'].str.split(expand = True)[0]
metadata_clean['rank'] = metadata_clean['rank'].str.replace(',', '')
metadata_clean['rank'] = pd.to_numeric(metadata_clean['rank'])

In [ ]:
metadata_clean.head(1)

,asin,rank,brand,price,also_view,also_buy,cate
0,6546546450,2938573,idea village,NaN,[],[],Beauty & Personal Care


## 資料切分

In [ ]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

In [ ]:
users_training = ratings_trainings['reviewerID'].unique()

## 產生推薦(item-based)

In [ ]:
import pandas as pd
import datetime
from itertools import combinations
from collections import defaultdict

def recommender(training_data, metadata, users=[], hot=False, k=10, days=9999, score=0):
    # loading data from dataframe
    # item_to_users dict:
    # {
    #   'item': {
    #       'user': ratings...
    #   }...
    # }

    training_data = training_data.sort_values("DATE", ascending=False).groupby(['reviewerID', 'asin']).head(1)
    training_data = training_data[(training_data['DATE'] + datetime.timedelta(days = days)) > '2018-09-01']
    ratehot_table = training_data.groupby(by = ['asin']).aggregate({'overall':['mean', 'size']})['overall']
    metadata = pd.merge(metadata, ratehot_table, on='asin', how='left')
    metadata = metadata.sort_values(['size'], ascending=False)
    metadata = metadata[(metadata['mean']<score) == False]

    hottest = metadata['asin'].tolist()[:k]
    
    if hot:
        return {user: hottest for user in users}

    item_to_users = defaultdict(dict)
    for _, row in training_data.iterrows():
        row = dict(row)
        user = row['reviewerID']
        item = row['asin']
        rating = float(row['overall'])
        if rating > score:
            item_to_users[item][user] = rating

    # print("data converted")

    user_to_items = defaultdict(dict)
    for item, rating_users in item_to_users.items():
        for user, rating in rating_users.items():
            user_to_items[user][item] = rating

    # print("data inverted")

    init_sim = lambda: [0, 0, 0]
    factory = lambda: defaultdict(init_sim)
    pre_item_similarity = defaultdict(factory)
    for user, items in user_to_items.items():
        if len(items) > 1:
            for i1, i2 in combinations(items.keys(), 2):
                xy = items[i1] * items[i2]
                xx = items[i1] ** 2
                yy = items[i2] ** 2
                pre_item_similarity[i1][i2][0] += xy
                pre_item_similarity[i1][i2][1] += xx
                pre_item_similarity[i1][i2][2] += yy

                pre_item_similarity[i2][i1][0] += xy
                pre_item_similarity[i2][i1][1] += xx
                pre_item_similarity[i2][i1][2] += yy

    # print("sim data prepared")

    item_similarity = {}
    for src_item in pre_item_similarity:
        item_similarity_order = []
        for dst_item, val in pre_item_similarity[src_item].items():
            xy = val[0]
            xx = val[1]
            yy = val[2]
            div = ((xx*yy) ** 0.5)
            if div == 0:
                continue
            similarity = xy / div
            if similarity < 0:
                continue
            for i, s in enumerate(item_similarity_order):
                target_similarity = s[1]
                if target_similarity < similarity:
                    item_similarity_order.insert(i, (dst_item, similarity))
                    break
            else:
                item_similarity_order.append((dst_item, similarity))
        item_similarity[src_item] = item_similarity_order

    print(f"get {k} recommendation items for user: {users}")

    recommendation = {}
    a = 0
    l = 0
    for user in users:
        items = []
        items_set = set()
        stop = False
        user_has_rated = set(user_to_items[user])
        for item in user_has_rated:
            if item in item_similarity:
                for sim_item, _ in item_similarity[item]:
                    # skip the item user has rated
                    if sim_item not in user_has_rated and sim_item not in items_set:
                        items.append(sim_item)
                        items_set.add(sim_item)
                    if len(items) >= k:
                        stop = True
                        break
                if stop:
                    break
        recommendation[user] = items
        if len(items) > 0:
            a += 1
            l += min(len(items), k)
    print(a, l)
    return recommendation    

## 結果評估(item-based)

In [ ]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    print(total)
    print(len(ratings_testings))
    return score

In [ ]:
ratings_by_user = recommender(ratings_trainings, metadata_clean, users, k=10, hot=False)

get 10 recommendation items for user: ['A100XQFWKQ30O2', 'A103T1QOGFCSEH', 'A106UKKSJ2KXPF', 'A10A7GV4D5A11V', 'A1119JJ37ZLB8R', 'A113UOOLBSZN52', 'A12M4U7WK4ALCR', 'A12T8YTW6VWT7S', 'A1364JXGKB46MM', 'A137DALOQFKBTI', 'A13FEZ3WV7S2EY', 'A13IV4I1B0RXMG', 'A13JU88JAHN72I', 'A13K55R6VH1OOD', 'A13P7VFU075A', 'A13SWYE4QLB6NG', 'A13ZTQ0Q4ATA41', 'A142EDN04OD62U', 'A142I22FIC8MZK', 'A14834QTII5TLT', 'A14A447VPACTBC', 'A14AP6MN5XO6LB', 'A14CLF25IX25US', 'A14LYXC3HTBAHI', 'A14VUW4KZ34EOE', 'A14Y32P26G9YL', 'A157T25PBS7MX4', 'A15HZDSERD85C8', 'A15JJ8J1FGADIX', 'A15ZCL70JXXH89', 'A1617KN2IAWZ6J', 'A16E0O88262HKA', 'A16NSZ58PTVIYF', 'A16UGDXRTDLJG5', 'A16X9HR3UFQQXY', 'A16Y7V1CZCWKFV', 'A174YOBOSW9WDN', 'A1786SKRAJXH86', 'A17K2BUZ20WD2', 'A17LYRFV645L0V', 'A18LNGVXDZBTUR', 'A19503XX7GU6J2', 'A19HVHRZDYFEOP', 'A19JM38B861BO4', 'A19KGY3W1P0MHT', 'A19KJPOF3QOJ24', 'A19U5RT4Q8Q1T6', 'A1AQNQUNX8B6JT', 'A1ARU0IN6RUFFD', 'A1BB4635KSKDMV', 'A1BCC7X6KRTBG0', 'A1BPJ56BXENC7', 'A1C0RI6W65FC8B', 'A1C60LJGZVE

In [ ]:
ratings_by_user = recommender(ratings_trainings, metadata_clean, users, k=10, hot=False, days=150, score=3)
evaluate(ratings_testings_by_user, ratings_by_user)

get 10 recommendation items for user: ['A100XQFWKQ30O2', 'A103T1QOGFCSEH', 'A106UKKSJ2KXPF', 'A10A7GV4D5A11V', 'A1119JJ37ZLB8R', 'A113UOOLBSZN52', 'A12M4U7WK4ALCR', 'A12T8YTW6VWT7S', 'A1364JXGKB46MM', 'A137DALOQFKBTI', 'A13FEZ3WV7S2EY', 'A13IV4I1B0RXMG', 'A13JU88JAHN72I', 'A13K55R6VH1OOD', 'A13P7VFU075A', 'A13SWYE4QLB6NG', 'A13ZTQ0Q4ATA41', 'A142EDN04OD62U', 'A142I22FIC8MZK', 'A14834QTII5TLT', 'A14A447VPACTBC', 'A14AP6MN5XO6LB', 'A14CLF25IX25US', 'A14LYXC3HTBAHI', 'A14VUW4KZ34EOE', 'A14Y32P26G9YL', 'A157T25PBS7MX4', 'A15HZDSERD85C8', 'A15JJ8J1FGADIX', 'A15ZCL70JXXH89', 'A1617KN2IAWZ6J', 'A16E0O88262HKA', 'A16NSZ58PTVIYF', 'A16UGDXRTDLJG5', 'A16X9HR3UFQQXY', 'A16Y7V1CZCWKFV', 'A174YOBOSW9WDN', 'A1786SKRAJXH86', 'A17K2BUZ20WD2', 'A17LYRFV645L0V', 'A18LNGVXDZBTUR', 'A19503XX7GU6J2', 'A19HVHRZDYFEOP', 'A19JM38B861BO4', 'A19KGY3W1P0MHT', 'A19KJPOF3QOJ24', 'A19U5RT4Q8Q1T6', 'A1AQNQUNX8B6JT', 'A1ARU0IN6RUFFD', 'A1BB4635KSKDMV', 'A1BCC7X6KRTBG0', 'A1BPJ56BXENC7', 'A1C0RI6W65FC8B', 'A1C60LJGZVE

0.001694915254237288

In [ ]:
ratings_by_user = recommender(ratings_trainings, metadata_clean, users, k=50, hot=False)
evaluate(ratings_testings_by_user, ratings_by_user)

data converted
data inverted
sim data prepared
get 50 recommendation items for user: ['A100XQFWKQ30O2', 'A103T1QOGFCSEH', 'A106UKKSJ2KXPF', 'A10A7GV4D5A11V', 'A1119JJ37ZLB8R', 'A113UOOLBSZN52', 'A12M4U7WK4ALCR', 'A12T8YTW6VWT7S', 'A1364JXGKB46MM', 'A137DALOQFKBTI', 'A13FEZ3WV7S2EY', 'A13IV4I1B0RXMG', 'A13JU88JAHN72I', 'A13K55R6VH1OOD', 'A13P7VFU075A', 'A13SWYE4QLB6NG', 'A13ZTQ0Q4ATA41', 'A142EDN04OD62U', 'A142I22FIC8MZK', 'A14834QTII5TLT', 'A14A447VPACTBC', 'A14AP6MN5XO6LB', 'A14CLF25IX25US', 'A14LYXC3HTBAHI', 'A14VUW4KZ34EOE', 'A14Y32P26G9YL', 'A157T25PBS7MX4', 'A15HZDSERD85C8', 'A15JJ8J1FGADIX', 'A15ZCL70JXXH89', 'A1617KN2IAWZ6J', 'A16E0O88262HKA', 'A16NSZ58PTVIYF', 'A16UGDXRTDLJG5', 'A16X9HR3UFQQXY', 'A16Y7V1CZCWKFV', 'A174YOBOSW9WDN', 'A1786SKRAJXH86', 'A17K2BUZ20WD2', 'A17LYRFV645L0V', 'A18LNGVXDZBTUR', 'A19503XX7GU6J2', 'A19HVHRZDYFEOP', 'A19JM38B861BO4', 'A19KGY3W1P0MHT', 'A19KJPOF3QOJ24', 'A19U5RT4Q8Q1T6', 'A1AQNQUNX8B6JT', 'A1ARU0IN6RUFFD', 'A1BB4635KSKDMV', 'A1BCC7X6KRTBG0', 

0.001694915254237288

In [ ]:
for s in range(6):
    ratings_by_user = recommender(ratings_trainings, metadata_clean, users, hot=False, score=s)
    print(s, evaluate(ratings_testings_by_user, ratings_by_user))

get 10 recommendation items for user: ['A100XQFWKQ30O2', 'A103T1QOGFCSEH', 'A106UKKSJ2KXPF', 'A10A7GV4D5A11V', 'A1119JJ37ZLB8R', 'A113UOOLBSZN52', 'A12M4U7WK4ALCR', 'A12T8YTW6VWT7S', 'A1364JXGKB46MM', 'A137DALOQFKBTI', 'A13FEZ3WV7S2EY', 'A13IV4I1B0RXMG', 'A13JU88JAHN72I', 'A13K55R6VH1OOD', 'A13P7VFU075A', 'A13SWYE4QLB6NG', 'A13ZTQ0Q4ATA41', 'A142EDN04OD62U', 'A142I22FIC8MZK', 'A14834QTII5TLT', 'A14A447VPACTBC', 'A14AP6MN5XO6LB', 'A14CLF25IX25US', 'A14LYXC3HTBAHI', 'A14VUW4KZ34EOE', 'A14Y32P26G9YL', 'A157T25PBS7MX4', 'A15HZDSERD85C8', 'A15JJ8J1FGADIX', 'A15ZCL70JXXH89', 'A1617KN2IAWZ6J', 'A16E0O88262HKA', 'A16NSZ58PTVIYF', 'A16UGDXRTDLJG5', 'A16X9HR3UFQQXY', 'A16Y7V1CZCWKFV', 'A174YOBOSW9WDN', 'A1786SKRAJXH86', 'A17K2BUZ20WD2', 'A17LYRFV645L0V', 'A18LNGVXDZBTUR', 'A19503XX7GU6J2', 'A19HVHRZDYFEOP', 'A19JM38B861BO4', 'A19KGY3W1P0MHT', 'A19KJPOF3QOJ24', 'A19U5RT4Q8Q1T6', 'A1AQNQUNX8B6JT', 'A1ARU0IN6RUFFD', 'A1BB4635KSKDMV', 'A1BCC7X6KRTBG0', 'A1BPJ56BXENC7', 'A1C0RI6W65FC8B', 'A1C60LJGZVE

## 產生推薦(user-based)

In [ ]:
import pandas as pd
import datetime
from itertools import combinations
from collections import defaultdict
# header: user_id,item_id,rating,timestamp

def recommender(training_data, metadata, users=[], hot=False, k=10, days=9999, score=0, remove_user=True):

    # loading data from dataframe
    # user_to_items dict:
    # {
    #   'user': {
    #       'item': ratings...
    #   }...
    # }
    training_data = training_data.sort_values("DATE", ascending=False).groupby(['reviewerID', 'asin']).head(1)
    training_data = training_data[(training_data['DATE'] + datetime.timedelta(days = days)) > '2018-09-01']
    ratehot_table = training_data.groupby(by = ['asin']).aggregate({'overall':['mean', 'size']})['overall']
    metadata = pd.merge(metadata, ratehot_table, on='asin', how='left')
    metadata = metadata.sort_values(['size'], ascending=False)
    metadata = metadata[(metadata['mean']<score) == False]

    hottest = metadata['asin'].tolist()[:k]

    if hot:
        return {user: hottest for user in users}

    user_to_items = defaultdict(dict)
    for _, row in training_data.iterrows():
        row = dict(row)
        user = row['reviewerID']
        item = row['asin']
        rating = float(row['overall'])

        if rating >= score:
            user_to_items[user][item] = rating

    print("total users before filtering: ", len(user_to_items))

    # remove obscure user to decrease data size
    # filtering params
    remove_obscure_user = remove_user
    user_rating_threshold = 3
    all_users = list(user_to_items.keys())
    for user in all_users:
        ratings = user_to_items[user]
        if remove_obscure_user and len(ratings) < user_rating_threshold:
            del user_to_items[user]

    print("total users after filtering: ", len(user_to_items))

    # generate item to user mapping dict
    # {
    #   'item': {
    #       'user': ratings...
    #   }...
    # }
    item_to_users = defaultdict(dict)
    for user, items in user_to_items.items():
        for item, rating in items.items():
            item_to_users[item][user] = rating

    # prepare data of computing user similarity 
    init_sim = lambda: [0 for _ in range(3)]
    factory = lambda: defaultdict(init_sim)
    pre_user_similarity = defaultdict(factory)
    n = len(item_to_users)
    index = 0
    for item, user_ratings in item_to_users.items():
        if len(user_ratings) > 1:
            # print(f"item: {item} have been rated by {len(user_ratings)} users progress: {index}/{n}")
            for user1, user2 in combinations(user_ratings.keys(), 2):
                xy = user_ratings[user1] * user_ratings[user2]
                xx = user_ratings[user1] ** 2
                yy = user_ratings[user2] ** 2
                pre_user_similarity[user1][user2][0] += xy
                pre_user_similarity[user1][user2][1] += xx
                pre_user_similarity[user1][user2][2] += yy

                pre_user_similarity[user2][user1][0] += xy
                pre_user_similarity[user2][user1][1] += xx
                pre_user_similarity[user2][user1][2] += yy
        index += 1

    user_similarity = {}
    for src_user in pre_user_similarity:
        user_similarity_order = []
        a = 0
        for dst_user, val in pre_user_similarity[src_user].items():
            a += 1
            xy = val[0]
            xx = val[1]
            yy = val[2]
            div = ((xx*yy) ** 0.5)
            if div == 0:
                continue
            similarity = xy / div
            if similarity < 0:
                continue
            for i, s in enumerate(user_similarity_order):
                target_similarity = s[1]
                if target_similarity < similarity:
                    user_similarity_order.insert(i, (dst_user, similarity))
                    break
            else:
                user_similarity_order.append((dst_user, similarity))
        user_similarity[src_user] = user_similarity_order
    
    recommendation = {}
    a = 0
    l = 0
    for user in users:
        if user in user_similarity:
            sim_users = user_similarity[user]
            recommended_items = []
            recommended_items_set = set()
            user_have_rated = set(user_to_items[user])
            stop_recommend = False
            for sim_user, _ in sim_users:
                items_from_sim_user = sorted(list(user_to_items[sim_user].items()), key=lambda item: item[1])
                for item, _ in items_from_sim_user:
                    if item not in user_have_rated and item not in recommended_items_set:
                        recommended_items.append(item)
                        recommended_items_set.add(item)
                    if len(recommended_items) >= k:
                        stop_recommend = True
                        break
                if stop_recommend:
                    break
            recommendation[user] = recommended_items
            a += 1
            l += len(recommended_items)
        elif hot:
            recommendation[user] = hottest
    print(a, l)            
    return recommendation

## 結果評估(user-based)

In [ ]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

In [ ]:
ratings_by_user = recommender(ratings_trainings, metadata_clean, users, hot=False)

total users before filtering:  323489
total users after filtering:  4793
3 26


In [ ]:
ratings_by_user = recommender(ratings_trainings, metadata_clean, users, hot=False, remove_user=False)

total users before filtering:  323489
total users after filtering:  323489


In [ ]:
scorelist = list(range(6))

for s in scorelist:
    ratings_by_user = recommender(ratings_trainings, metadata_clean, users, hot=False, days=180, score=s)
    print(s, evaluate(ratings_testings_by_user, ratings_by_user))

total users before filtering:  18662
total users after filtering:  111
0 0.0
total users before filtering:  18662
total users after filtering:  111
1 0.0
total users before filtering:  16034
total users after filtering:  89
2 0.0
total users before filtering:  14905
total users after filtering:  79
3 0.0
total users before filtering:  13454
total users after filtering:  73
4 0.0
total users before filtering:  11438
total users after filtering:  69
5 0.0


In [ ]:
scorelist = list(range(6))

for s in scorelist:
    ratings_by_user = recommender(ratings_trainings, metadata_clean, users, hot=True, days=180, score=s)
    print(s, evaluate(ratings_testings_by_user, ratings_by_user))

0 0.09830508474576272
1 0.09830508474576272
2 0.09830508474576272
3 0.09830508474576272
4 0.01694915254237288
5 0.006779661016949152


In [ ]:
daylist = [15, 30, 60, 90, 180, 360]

for d in daylist:
    ratings_by_user = recommender(ratings_trainings, metadata_clean, users, hot=True, days=d, score=3)
    print(d, evaluate(ratings_testings_by_user, ratings_by_user))

15 0.1423728813559322
30 0.15423728813559323
60 0.15084745762711865
90 0.13389830508474576
180 0.09830508474576272
360 0.09830508474576272


## 產生推薦(surprise套件)

In [ ]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 5.5 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619450 sha256=d79c1921a9c75068846e5ca4ccb6e6f93b1c34860ba9e800c4c134517977afb5
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
import time
import datetime
import pandas as pd
from surprise import Reader
from surprise import Dataset
from surprise import KNNBasic

def recommender(training_data, users=[], k=10, user_based=False, algo=KNNBasic, days=9999, score=0):

    training_data = (
        training_data
        .sort_values("DATE", ascending=False)
        .groupby(['reviewerID', 'asin']).head(1)
    )
    training_data = training_data[(training_data['DATE'] + datetime.timedelta(days = days)) > '2018-09-01']
    training_data = training_data[training_data['overall'] > score]

    reader = Reader(rating_scale=(0, 5))
    training_data = training_data[['reviewerID', 'asin', 'overall']]
    data = Dataset.load_from_df(training_data, reader=reader)

    sim_options = {
        'name': 'cosine',
        'user_based': user_based  # compute similarities between items
    }
    algo_impl = algo(sim_options=sim_options)
    trainset = data.build_full_trainset()
    algo_impl.fit(trainset)

    recommendation = {}
    a = 0
    l = 0
    for user in users:
        items_user_rated = set(training_data.loc[training_data['reviewerID'] == user]['asin'].to_list())
        recommend_item_list = []
        recommend_item_set = set()
        for item in items_user_rated:
            iid = algo_impl.trainset.to_inner_iid(item)
            recommend_items_iid = algo_impl.get_neighbors(iid, k)
            for sim_item_iid in recommend_items_iid:
                item_raw_id = algo_impl.trainset.to_raw_iid(sim_item_iid)
                if item_raw_id not in items_user_rated and item_raw_id not in recommend_item_set:
                    recommend_item_list.append(item_raw_id)
                    recommend_item_set.add(item_raw_id)

            if len(recommend_item_list) >= k:
                recommend_item_list = recommend_item_list[:k]
                break
        recommendation[user] = recommend_item_list
        if len(recommend_item_list) > 0:
            a += 1
            l += len(recommend_item_list)
    print(a, l)
    return recommendation

## 結果評估(surprise套件)

In [ ]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

In [ ]:
ratings_by_user = recommender(ratings_trainings, users)

Computing the cosine similarity matrix...


In [ ]:
for s in range(6):
    ratings_by_user = recommender(ratings_trainings, users, days=30, score=s)
    print(s, evaluate(ratings_testings_by_user, ratings_by_user))

Computing the cosine similarity matrix...
Done computing similarity matrix.
0 0.0
Computing the cosine similarity matrix...
Done computing similarity matrix.
1 0.0
Computing the cosine similarity matrix...
Done computing similarity matrix.
2 0.0
Computing the cosine similarity matrix...
Done computing similarity matrix.
3 0.0
Computing the cosine similarity matrix...
Done computing similarity matrix.
4 0.0
Computing the cosine similarity matrix...
Done computing similarity matrix.
5 0.0


In [ ]:
for d in range(30,181,30):
    ratings_by_user = recommender(ratings_trainings, users, days=d, score=3)
    print(d, evaluate(ratings_testings_by_user, ratings_by_user))

Computing the cosine similarity matrix...
Done computing similarity matrix.
30 0.0
Computing the cosine similarity matrix...
Done computing similarity matrix.
60 0.0
Computing the cosine similarity matrix...
Done computing similarity matrix.
90 0.0
Computing the cosine similarity matrix...
Done computing similarity matrix.
120 0.0
Computing the cosine similarity matrix...
Done computing similarity matrix.
150 0.001694915254237288
Computing the cosine similarity matrix...
Done computing similarity matrix.
180 0.001694915254237288
